In [1]:
target: str = ''
start: str = '2025-01-01'
end: str = '2025-12-31'

In [2]:
import json
from datetime import date

import petl as etl
from utils.paths import PUBLISHED, SITE

In [3]:
TARGET = SITE / 'dashboard/kpi' / target / '_data'

In [4]:
TARGET.mkdir(exist_ok=True, parents=True)

In [5]:
date_range = (
    date.fromisoformat(start),
    min(date.today(), date.fromisoformat(end))
)

In [6]:
with open(TARGET / 'timestamp.json', 'w') as f:
    json.dump(dict(zip(['start', 'end'], (str(x) for x in date_range))), f)

In [7]:
data = (
    etl
    .fromjson(PUBLISHED / 'programme/projects.json')
    .convert(['Date From', 'Date To'], etl.dateparser('%Y-%m-%d'))
    .selectge('Date To', date_range[0])
    .selectle('Date From', date_range[1])
)

In [8]:
def rowgenerator(row):
    for v in row['Producing model (from Audiences Sync)']:
        yield [v]

producing_model = etl.cat(
    data
    .rowmapmany(rowgenerator, header=['Producing model'])
    .aggregate('Producing model', len),

    data
    .selecteq('Producing model (from Audiences Sync)', [])
    .aggregate(None, len)
    .addfield('Producing model', 'UNKNOWN')
)

In [9]:
projects = {
    'count': data.nrows(),
    'producing_model': dict(producing_model.records())
}

In [10]:
with open(TARGET / 'events.json', 'w') as f:
    json.dump(
        {
            'projects': projects
        },
        f,
        indent=2
    )